In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2


In [ ]:
df_sub = pd.read_csv('sample_submission.csv')
df_test = pd.read_csv('new_test/new_test.csv')

In [ ]:
X_train = []
y_train = []




for i in os.listdir('Train/Train/'):
    for j in os.listdir(f'Train/Train/{i}'):
        try:        
            image = cv2.imread(f'Train/Train/{i}/{j}')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = image.reshape(64, 64, 1)
            image = image / 255.0
            X_train.append(image)
            y_train.append(i)
        except:
            continue


X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
test_pixels = df_test.values

X_test = test_pixels.reshape(-1, 64, 64, 1)

X_test = X_test.astype('float32') / 255.0


In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
len(os.listdir('Train/Train/'))

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train, num_classes=78)

In [ ]:
df_test

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(78, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

history = model.fit(
    X_train_split, y_train_split,
    epochs=20, 
    batch_size=32,
    validation_data=(X_val, y_val)
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.save('model.keras')